# Aula 06: WebScraping

Um dos maiores desafios de Ciência de Dados é conseguir os Dados. 

Uma forma é conseguir dados com o cliente/sua empresa e analisá-los.

Uma outra forma de conseguir é através de **[APIs](https://codeburst.io/6-interesting-apis-to-check-out-in-2018-5d6830063f29)**, interfaces para se conseguir dados oficialmente de fontes confiáveis. 

O **Facebook** tinha uma ótima API. Tinha. Depois do escândalo da Cambridge Analytica, boa parte das funções interessantes foi tirada.

O **[Instagram](https://www.instagram.com/developer/)**, **[Twitter](https://developer.twitter.com/en/docs.html)** e a **[NASA](https://api.nasa.gov/)** tem ótimas APIs para diversas aplicações, recomendo ir atrás delas.

Quando isso não é possível (a fonte não tem uma API, ou os dados estão **desestruturados** e em *sites*, existe o **WebScraping**!

#### Os algoritmos de WebScraping fazem um *request*, pegam o código-fonte da página e buscam a informação que você pedir nele
---

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs

Vamos buscar informações sobre o histórico de uso de um site, através do site [Rank2Traffic](https://www.rank2traffic.com/).

In [2]:
url = 'https://www.rank2traffic.com/stackoverflow.com' # Because Stack Overflow is God.

In [3]:
session = req.Session() # Começando a sessão, como se estivesse abrindo o browser
# Os headers fazem o Python fingir ser um browser comum, para evitar que o site bloqueie ele
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8"}
ref = session.get(url, headers=headers) # Aqui ele abre o site e puxa o código-fonte
soup = bs(ref.text, "html.parser") # E transforma o código em texto (método .text) e logo depois em um objeto BeautifulSoup
print(soup) # que tem essa cara


<!DOCTYPE html>

<html lang="en">
<head><meta charset="utf-8"/>
<script data-cfasync="false" data-pagespeed-no-defer="">var __ez=__ez||{};__ez.evt={};__ez.ck={};__ez.dot={};__ez.template={};__ez.template.isOrig=false;__ez.evt.add=function(e,t,n){e.addEventListener?e.addEventListener(t,n,!1):e.attachEvent?e.attachEvent("on"+t,n):e["on"+t]=n()},__ez.evt.remove=function(e,t,n){e.removeEventListener?e.removeEventListener(t,n,!1):e.detachEvent?e.detachEvent("on"+t,n):delete e["on"+t]};__ez.ck.get=function(e,n){null!==n&&(e=e+"_"+n);for(var o=e+"=",t=decodeURIComponent(document.cookie).split(";"),i=0;i<t.length;i++){for(var c=t[i];" "===c.charAt(0);)c=c.substring(1);if(0===c.indexOf(o))return c.substring(o.length,c.length)}return""},__ez.ck.setByCat=function(e,n){if("undefined"!=typeof cmpIsOn){if(null!=n){var o=__ez.ck.get("ezCMPCookieConsent",null);-1!==(o=o.substring(1,o.length)).indexOf(n+"=1")?document.cookie=e:""===o&&"undefined"!=typeof cmpCookies&&(void 0===cmpCookies[n]&&(cmpCookie

In [4]:
# Extraindo a tabela
tab = soup.find('table', {'id':"myTab1-table"}) # método .find: encontre a primeira tabela com id 'myTab1-table'
rows=list() # para salvar as linhas da tabela
for row in tab.findAll("tr"):
    try:
        rows.append(row.findAll("td")[1].string.replace(" ","").replace("\n",""))# a segunda coluna (índice 1) é a que interessa
    except IndexError: # na primeira linha, não tem índice 1. então coloque o nome do site 
        rows.append(url[-17:])

In [5]:
# Extraindo dados complementares de uso
meta = soup.findAll('div',{'class':'infobox-data'})[1:] # método .findAll: Encontre TODOS nessas condições, e retorne uma lista
for box in meta:
    rows.append(box.find('span').string)
    rows.append(box.find('div').string)

In [6]:
# Extraindo percentual de acessos por país
tab = soup.find('table', {'class':"table table-hover table-condensed"})
try: 
    for row in tab.findAll("tr"):
        try:
            rows.append(row.findAll("td")[0].string)
            rows.append(row.findAll("td")[1].string)
        except IndexError:
            rows.append('Paises')
except AttributeError:
    rows.append('Supondo, sem info oficial')
    rows.append('100%')

In [7]:
rows

['stackoverflow.com',
 '352000',
 '1850000',
 '5820000',
 '8230000',
 '10500000',
 '12900000',
 '23500000',
 '29500000',
 '35100000',
 '40000000',
 '41300000',
 '44000000',
 '44300000',
 '49800000',
 '57100000',
 '58600000',
 '59200000',
 '62600000',
 '64800000',
 '71300000',
 '73400000',
 '75400000',
 '76600000',
 '76800000',
 '84500000',
 '93100000',
 '99400000',
 '94700000',
 '127000000',
 '150000000',
 '153000000',
 '156000000',
 '152000000',
 '154000000',
 '171000000',
 '163000000',
 '167000000',
 '186000000',
 '174000000',
 '169000000',
 '188000000',
 '200000000',
 '209000000',
 '205000000',
 '190000000',
 '181000000',
 '169000000',
 '164000000',
 '164000000',
 '164000000',
 '171000000',
 '175000000',
 '192000000',
 '202000000',
 '207000000',
 '209000000',
 '216000000',
 '220000000',
 '221000000',
 '222000000',
 '218000000',
 '214000000',
 '240000000',
 '254000000',
 '266000000',
 '273000000',
 '302000000',
 '316000000',
 '303000000',
 '297000000',
 '296000000',
 '295000000',
 '2

In [8]:
import pandas as pd
df = pd.DataFrame(rows)
df.to_excel("StackOverflow.xlsx")